In [11]:
# Set up the Endpoint from a prior deployment
scoring_uri = 'http://5e070320-2fcf-4b74-b0c5-00c728febae3.northcentralus.azurecontainer.io/score'
scoring_uri

'http://5e070320-2fcf-4b74-b0c5-00c728febae3.northcentralus.azurecontainer.io/score'

In [12]:
import requests
import json

# Test the endpoint by having the Blue Jackets host the Rangers in the regular season
data = {
  "Inputs": {
    "data": [{"type": "R", "homeTeam": "Blue Jackets", "awayTeam": "Rangers"}]
  },
  "GlobalParameters": 1.0
}

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
results = resp.json()['Results']
results

[25.08131977502827]

## Evaluating all Matchups

In [16]:
import pandas as pd

# We need a list of teams to iterate through
df = pd.read_csv('data/team_info.csv')
teams = df['teamName'].unique()
teams

array(['Devils', 'Flyers', 'Kings', 'Lightning', 'Bruins', 'Rangers',
       'Penguins', 'Red Wings', 'Sharks', 'Predators', 'Canucks',
       'Blackhawks', 'Senators', 'Canadiens', 'Wild', 'Capitals', 'Blues',
       'Ducks', 'Coyotes', 'Islanders', 'Maple Leafs', 'Panthers',
       'Sabres', 'Flames', 'Avalanche', 'Stars', 'Blue Jackets', 'Jets',
       'Oilers', 'Golden Knights', 'Hurricanes', 'Thrashers'],
      dtype=object)

In [26]:
# Loop over all home and away matchups to build a list of the penalties we should expect
matchups = []
scenarios = []
for homeTeam in teams:
  for awayTeam in teams:

    # Don't allow teams to play themselves
    if homeTeam == awayTeam:
      continue

    matchups.append({
        "homeTeam": homeTeam,
        "awayTeam": awayTeam
    })
    scenarios.append(      
      {
        "type": "R", # Regular Season
        "homeTeam": homeTeam,
        "awayTeam": awayTeam
      }) 
    scenarios.append(      
      {
        "type": "P", # Playoff game
        "homeTeam": homeTeam,
        "awayTeam": awayTeam
      }) 

scenarios

[{'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Flyers'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Flyers'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Kings'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Kings'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Lightning'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Lightning'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Bruins'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Bruins'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Rangers'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Rangers'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Penguins'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Penguins'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Red Wings'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Red Wings'},
 {'type': 'R', 'homeTeam': 'Devils', 'awayTeam': 'Sharks'},
 {'type': 'P', 'homeTeam': 'Devils', 'awayTeam': 'Sharks'},
 {'type': 'R', 'homeTeam

In [21]:
# Make a request containing every scenario we care about
data = {
  "Inputs": {
    "data": scenarios
  },
  "GlobalParameters": 1.0
}

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
results = resp.json()['Results']
results

[36.62266958177161,
 41.624281272772464,
 25.217129888228953,
 35.006036139699304,
 31.969213994861942,
 55.00140904149908,
 27.532368067537135,
 37.27153410864529,
 40.227946278691014,
 44.142897833532174,
 25.968482849683795,
 29.745796324000736,
 26.583428102744097,
 40.89242678388668,
 33.42623618499357,
 36.954670222238505,
 26.16736825733506,
 33.256342294886124,
 42.6601446758536,
 57.367306402197755,
 32.74550304412605,
 37.1010850248512,
 29.013515434503375,
 37.700836574296886,
 28.37362540082322,
 35.805206356930256,
 25.29124309362573,
 36.642401719962834,
 31.853486518956267,
 37.72818128981341,
 23.508598097928775,
 33.15417498552522,
 29.10739038906826,
 37.657065705953315,
 25.188080416314527,
 35.586859045523255,
 24.043146097772386,
 35.1976131691792,
 23.47259207246472,
 31.34612892823621,
 29.080321808963525,
 39.012543623891005,
 24.77956864869865,
 35.09825206545614,
 27.649841232728136,
 37.33016530072389,
 36.58289441060868,
 45.350992111414214,
 28.961196568486

In [25]:
# Aggregate the predictions into the matchups objects
i = 0
for matchup in matchups:
    matchup['penaltyMinutesRegularSeason'] = results[i]
    matchup['penaltyMinutesPlayoffs'] =  results[i + 1]
    i += 2

# Move our matchup predictions to a dataframe and save them for visualization later
df_outcomes = pd.DataFrame(matchups)
df_outcomes.to_csv('data/matchup_predictions.csv')

# Display the top few rows
df_outcomes.head()

,homeTeam,awayTeam,penaltyMinutesRegularSeason,penaltyMinutesPlayoffs
0,Devils,Flyers,36.622670,41.624281
1,Devils,Kings,25.217130,35.006036
2,Devils,Lightning,31.969214,55.001409
3,Devils,Bruins,27.532368,37.271534
4,Devils,Rangers,40.227946,44.142898
